In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from qiskit import IBMQ
import sklearn.datasets as skd
from sklearn.model_selection import train_test_split
from qiskit.aqua.components.optimizers import SPSA
from qiskit.circuit.library import ZZFeatureMap, TwoLocal
from qiskit.aqua.algorithms import QSVM, VQC
from qiskit.providers.ibmq import least_busy
from qiskit import Aer
from qiskit.aqua import QuantumInstance
from sklearn.metrics import classification_report
from sklearn.preprocessing import normalize
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute
from sklearn.datasets import make_blobs
from imblearn.under_sampling import RandomUnderSampler
import math
import funcs
from sklearn.datasets import make_classification, make_moons, make_circles, make_blobs
#https://www.kaggle.com/merishnasuwal/breast-cancer-prediction-dataset

In [2]:
# Create Fake dataset

# X, y = make_circles(n_samples=30, shuffle=True, noise=None, random_state=None, factor=0.1)
# X,y = make_classification(n_samples=30, n_features=2, n_informative=2, n_redundant=0, 
#                            n_repeated=0, n_classes=2, random_state=6, class_sep=0.2)
X,y = make_blobs(n_samples=200, centers=2, n_features=5, center_box=[- 50.0, 50.0], random_state=42)
# X,y = skd.make_gaussian_quantiles(mean=None, cov=1.0, n_samples=30, n_features=2, n_classes=2, shuffle=True, random_state=None)
# X,y =skd.make_moons(n_samples=30, shuffle=True, noise=False, random_state=42)
# X, y = make_moons(n_samples=30, shuffle=True, noise=None, random_state=None)

In [3]:
X.shape

(200, 5)

In [4]:
# X = X.values
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [5]:
y

array([1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 0])

In [6]:
def nextPowerOf2(n):
    p = 1
    if (n and not(n & (n - 1))):
        return n
    while (p < n) :
        p <<= 1     
    return p;

In [7]:
# Rotator
from typing import Callable, List, Optional, Tuple, Union
from math import atan2
def rotate_matrix_nd(bs,degrees):
    ang = np.deg2rad(degrees)
    dim = np.shape(bs)[0]
    Rot_mat = np.eye(dim)
    for k in range(0,dim-2):
        for l in range(dim-1,k,-1):
            a = atan2(bs[l,k],bs[l-1,k])
            R = np.eye(dim)
            R_temp = np.array([[np.cos(a),np.sin(a)],[-1*np.sin(a),np.cos(a)]])
            tmp = l - 1
            R[tmp:tmp+R_temp.shape[0],tmp:tmp+R_temp.shape[0]] = R_temp
            bs=np.matmul(R,bs)
            Rot_mat = np.matmul(R,Rot_mat)
    R = np.eye(dim)
    R[dim-2:dim,dim-2:dim] = np.array([[np.cos(ang),-1*np.sin(ang)],[np.sin(ang),np.cos(ang)]])
    Rot_mat = np.matmul(np.linalg.inv(Rot_mat),np.matmul(R,Rot_mat))
    return Rot_mat

In [8]:
dim = 5
# v = np.logical_or(np.eye(dim),np.fliplr(np.eye(dim))).astype(int)
# v = v[:,:-2]
v = np.eye(dim)[:,:-2]
v[-2][0] = 1
v[-1][1] = 1

In [9]:
# for x in range(0,195,10):
#     print(-x)


# VQC - Amplitude Encoding Mottonen Method

In [13]:
# number of qubits
n = 3
num_qubits = n # needed for D&C
# set random seed
np.random.seed(42)
# number of samples
num_data = len(y) # 100
# number of training samples
num_train = int(0.75*num_data) # 75
# number of parameterized layers
layers = 20 #5
batch_size = 50#20
iterations = 30#20
# initial parameters
params_init = np.random.randn(layers, n, 3)*0.01
bias_init = 0.01
learning_rate = 0.01
momentum = 0.05
# set backend to run 
backend = Aer.get_backend('qasm_simulator')
    
for angul in range(0,20,10):
    print("Voy en "+str(angul))
    rot = rotate_matrix_nd(v,-angul)
    XX = np.empty((0,dim))
    for point in X:
        tmp_point = np.matmul(rot,np.atleast_2d(point).T)
        tmp_point = np.atleast_2d(tmp_point).T
        XX = np.append(XX, tmp_point,axis=0)
    
    X = XX
    padding_X = 0.3*np.ones((len(X), nextPowerOf2(len(X[0]))-len(X[0])))
    X_pad = np.c_[np.c_[X, padding_X], np.zeros((len(X), 0))]
    X_pad
#     print("First X sample (original)  :", X_pad[0])
    # normalize each input
    normalization = np.sqrt(np.sum(X**2, -1))
    X_norm = (X_pad.T / normalization).T
#     print("First X sample (normalized):", X_norm[0])

    # angles for state preparation are new features; impute nans
    features = np.array([funcs.get_angles(x, n) for x in X_norm])
    features = np.nan_to_num(features)
    feats_train, feats_val, Y_train, Y_val = train_test_split(features, y, test_size=0.4,  shuffle=True, random_state=42)
    
    # train model
    var, bias, Y_val, pred_val = funcs.train_model(n, layers, params_init, bias_init, batch_size, learning_rate, momentum, iterations, feats_train, Y_train, feats_val, Y_val, features, y,angul)
    

Voy en 0
Voy en 10


# Classic SVM

In [10]:
from sklearn import svm
sample_train, sample_test, label_train, label_test = train_test_split(X, y, test_size=0.10,  shuffle=True, random_state=42)

In [11]:
clf = svm.SVC(gamma='scale')
clf.fit(sample_train, label_train)

SVC()

In [12]:
print(classification_report(label_test, clf.predict(sample_test) , zero_division=0))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       1.00      1.00      1.00        11

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



KeyboardInterrupt: 

In [28]:
print(classification_report(Y_val, pred_val))

              precision    recall  f1-score   support

           0       0.49      0.89      0.63        35
           1       0.69      0.22      0.33        41

    accuracy                           0.53        76
   macro avg       0.59      0.55      0.48        76
weighted avg       0.60      0.53      0.47        76



In [353]:
Y_val

array([0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1])

In [354]:
pred_val

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1])